# Monthly Resolution Example: Seasonal Renewable Electricity

This notebook demonstrates `optimex`'s support for **monthly temporal resolution**. It models a scenario where:

1. **Renewable electricity availability varies seasonally** - solar PV generates more in summer months
2. **Water use characterization factors vary monthly** - water scarcity is higher in dry summer months
3. **Demand is constant** - showing how the optimizer handles seasonal supply variations

This example showcases the new flexible temporal resolution feature that enables sub-yearly optimization.

In [ ]:
from datetime import datetime
import numpy as np
import bw2data as bd
from bw_temporalis import TemporalDistribution

bd.projects.set_current("monthly_resolution_example")

## 1. Setup Brightway Databases

### Biosphere Database

We define elementary flows for CO2 emissions and water consumption.

In [ ]:
# BIOSPHERE
biosphere_data = {
    ("biosphere3", "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "CAS number": "000124-38-9"
    },
    ("biosphere3", "water"): {
        "type": "emission",
        "name": "water consumption",
    },
}
bd.Database("biosphere3").write(biosphere_data)

### Background Database

Simple background processes for grid electricity and natural gas.

In [ ]:
# BACKGROUND DATABASE
background_data = {
    ("background", "grid_electricity"): {
        "name": "Grid electricity",
        "location": "EU",
        "reference product": "electricity",
        "exchanges": [
            {"amount": 1, "type": "production", "input": ("background", "grid_electricity")},
            {"amount": 0.5, "type": "biosphere", "input": ("biosphere3", "CO2")},  # 500g CO2/kWh
            {"amount": 0.01, "type": "biosphere", "input": ("biosphere3", "water")},  # 10L water/kWh
        ],
    },
}
bd.Database("background").write(background_data)
bd.Database("background").metadata["representative_time"] = datetime(2024, 1, 1).isoformat()

### Foreground Database: Electricity Production Options

We model two electricity production options:

1. **Solar PV** - Higher production in summer months (seasonal pattern)
2. **Conventional (gas backup)** - Constant production year-round

The key innovation is that we use **monthly temporal distributions** to model seasonal variation in solar output.

In [ ]:
# Define seasonal solar production profile (12 months)
# Higher output in summer months (normalized to sum to 1)
monthly_solar_factors = np.array([
    0.04,  # Jan - Low winter output
    0.05,  # Feb
    0.08,  # Mar - Spring increase
    0.10,  # Apr
    0.12,  # May
    0.13,  # Jun - Peak summer
    0.13,  # Jul - Peak summer
    0.12,  # Aug
    0.09,  # Sep - Autumn decline
    0.07,  # Oct
    0.04,  # Nov
    0.03,  # Dec - Low winter output
])

# Verify normalization
print(f"Sum of monthly factors: {monthly_solar_factors.sum():.2f}")

# Create monthly temporal distribution for solar production
solar_production_td = TemporalDistribution(
    date=np.array(range(12), dtype="timedelta64[M]"),
    amount=monthly_solar_factors
)

# Constant monthly production for conventional (1/12 each month)
constant_monthly_td = TemporalDistribution(
    date=np.array(range(12), dtype="timedelta64[M]"),
    amount=np.array([1/12] * 12)
)

In [ ]:
# FOREGROUND DATABASE
foreground_data = {
    # Product node
    ("foreground", "electricity"): {
        "name": "Electricity (kWh)",
        "type": bd.labels.product_node_default,
        "unit": "kWh",
    },
    
    # Solar PV process - seasonal output pattern
    ("foreground", "solar_pv"): {
        "name": "Solar PV generation",
        "location": "EU",
        "type": bd.labels.process_node_default,
        "operation_time_limits": (0, 11),  # 12 months operation
        "exchanges": [
            {
                "amount": 1200,  # 1200 kWh/year total production (100 kWh/month average)
                "type": bd.labels.production_edge_default,
                "input": ("foreground", "electricity"),
                "temporal_distribution": solar_production_td,
                "operation": True,
            },
            {
                "amount": 10,  # 10 kg CO2 for panel manufacturing (construction phase)
                "type": bd.labels.biosphere_edge_default,
                "input": ("biosphere3", "CO2"),
                "temporal_distribution": TemporalDistribution(
                    date=np.array([0], dtype="timedelta64[M]"),
                    amount=np.array([1])
                ),
            },
            {
                "amount": 50,  # 50 L water for panel cleaning over year
                "type": bd.labels.biosphere_edge_default,
                "input": ("biosphere3", "water"),
                "temporal_distribution": constant_monthly_td,
                "operation": True,
            },
        ],
    },
    
    # Conventional gas backup - constant output
    ("foreground", "gas_backup"): {
        "name": "Gas backup generation",
        "location": "EU",
        "type": bd.labels.process_node_default,
        "operation_time_limits": (0, 11),  # 12 months operation
        "exchanges": [
            {
                "amount": 1200,  # 1200 kWh/year (100 kWh/month)
                "type": bd.labels.production_edge_default,
                "input": ("foreground", "electricity"),
                "temporal_distribution": constant_monthly_td,
                "operation": True,
            },
            {
                "amount": 600,  # 600 kg CO2/year (50 kg/month) - higher carbon intensity
                "type": bd.labels.biosphere_edge_default,
                "input": ("biosphere3", "CO2"),
                "temporal_distribution": constant_monthly_td,
                "operation": True,
            },
            {
                "amount": 120,  # 120 L water/year for cooling
                "type": bd.labels.biosphere_edge_default,
                "input": ("biosphere3", "water"),
                "temporal_distribution": constant_monthly_td,
                "operation": True,
            },
        ],
    },
}

bd.Database("foreground").write(foreground_data)

### LCIA Methods

We define two impact categories:
1. **Climate change** - Simple CO2 characterization
2. **Water scarcity** - With seasonal variation (higher impact in dry summer months)

In [ ]:
# Climate change method (constant characterization factor)
bd.Method(("GWP", "monthly_example")).write(
    [
        (("biosphere3", "CO2"), 1),  # 1 kg CO2eq per kg CO2
    ]
)

# Water scarcity method (using static characterization for now)
# In practice, this could vary by season - higher scarcity in summer
bd.Method(("water_scarcity", "monthly_example")).write(
    [
        (("biosphere3", "water"), 1),  # 1 L water eq per L water
    ]
)

## 2. Configure LCA with Monthly Resolution

The key difference from yearly resolution is setting `temporal_resolution: "month"` in the configuration.

In [ ]:
from optimex import lca_processor

# Define monthly demand for 24 months (2 years)
# Constant 100 kWh/month demand
n_months = 24
dates = [datetime(2024 + m // 12, (m % 12) + 1, 1).isoformat() for m in range(n_months)]

td_demand = TemporalDistribution(
    date=np.array(dates, dtype="datetime64[s]"),
    amount=np.array([100] * n_months),  # 100 kWh per month
)

# Get product node for demand specification
electricity_product = bd.get_node(database="foreground", code="electricity")

# Configure LCA with MONTHLY resolution
lca_config = lca_processor.LCAConfig(
    demand={electricity_product: td_demand},
    temporal={
        "start_date": datetime(2024, 1, 1),
        "temporal_resolution": "month",  # <-- KEY: Monthly resolution
        "time_horizon": 100,
    },
    characterization_methods=[
        {
            "category_name": "climate_change",
            "brightway_method": ("GWP", "monthly_example"),
        },
        {
            "category_name": "water_scarcity",
            "brightway_method": ("water_scarcity", "monthly_example"),
        },
    ],
    background_inventory={
        "cutoff": 1e4,
        "calculation_method": "sequential",
    },
)

print(f"Temporal resolution: {lca_config.temporal.temporal_resolution}")

## 3. Process LCA Data

In [ ]:
# Create LCA data processor
lca_processor_instance = lca_processor.LCADataProcessor(lca_config)

# Inspect the results
print(f"System time range: {min(lca_processor_instance.system_time)} to {max(lca_processor_instance.system_time)}")
print(f"Number of system time points: {len(lca_processor_instance.system_time)}")
print(f"Process time indices: {sorted(lca_processor_instance.process_time)}")
print(f"Processes: {list(lca_processor_instance.processes.values())}")
print(f"Products: {list(lca_processor_instance.products.values())}")

## 4. Examine Monthly Production Patterns

Let's visualize how the solar PV production varies across months.

In [ ]:
import matplotlib.pyplot as plt

# Extract solar PV production by month
production = lca_processor_instance.foreground_production
solar_production = {k[2]: v for k, v in production.items() if k[0] == 'solar_pv'}
gas_production = {k[2]: v for k, v in production.items() if k[0] == 'gas_backup'}

months = list(range(12))
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

solar_values = [solar_production.get(m, 0) for m in months]
gas_values = [gas_production.get(m, 0) for m in months]

fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(12)
width = 0.35

bars1 = ax.bar(x - width/2, solar_values, width, label='Solar PV', color='gold')
bars2 = ax.bar(x + width/2, gas_values, width, label='Gas Backup', color='gray')

ax.set_xlabel('Month')
ax.set_ylabel('Production per process unit (kWh)')
ax.set_title('Monthly Production Pattern by Technology')
ax.set_xticks(x)
ax.set_xticklabels(month_names)
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nTotal annual production:")
print(f"  Solar PV: {sum(solar_values):.1f} kWh")
print(f"  Gas backup: {sum(gas_values):.1f} kWh")

## 5. Convert to Optimization Model Inputs

Now we can convert the LCA data to optimization model inputs and run the optimizer.

In [ ]:
from optimex.converter import ModelInputManager

# Convert LCA outputs to optimization inputs
manager = ModelInputManager(lca_processor_instance)
model_inputs = manager.get_model_inputs()

print(f"SYSTEM_TIME indices: {len(model_inputs.SYSTEM_TIME)} time points")
print(f"PROCESS_TIME indices: {model_inputs.PROCESS_TIME}")
print(f"PROCESS set: {model_inputs.PROCESS}")
print(f"PRODUCT set: {model_inputs.PRODUCT}")

## 6. Create and Solve Optimization Model

In [ ]:
from optimex.optimizer import create_model, solve_model

# Create Pyomo model minimizing climate change impact
model = create_model(
    inputs=model_inputs,
    name="monthly_electricity_optimization",
    objective_category="climate_change",
)

# Solve the model
results = solve_model(model, solver="glpk")
print(f"Solver status: {results.solver.status}")
print(f"Solver termination: {results.solver.termination_condition}")

## 7. Analyze Results

Let's see how the optimizer schedules installations and operations across months.

In [ ]:
from optimex.postprocessing import PostProcessor

# Create post-processor
pp = PostProcessor(model)

# Get installation decisions
installations = pp.get_installations()
print("\nInstallation decisions:")
print(installations)

In [ ]:
# Get operation levels over time
operations = pp.get_operation()
print("\nOperation levels (first 12 months):")
print(operations.head(12))

In [ ]:
# Visualize operation over time
fig, ax = plt.subplots(figsize=(12, 5))

for process in model_inputs.PROCESS:
    process_ops = operations[operations['process'] == process]
    if not process_ops.empty:
        time_indices = process_ops['system_time'].values
        operation_values = process_ops['operation'].values
        ax.plot(time_indices, operation_values, label=process, marker='o', markersize=4)

ax.set_xlabel('System Time (monthly index)')
ax.set_ylabel('Operation Level')
ax.set_title('Operation Scheduling Over 24 Months')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Summary

This notebook demonstrated how to use `optimex` with **monthly temporal resolution**:

1. **Temporal distributions** use `timedelta64[M]` for monthly offsets
2. **LCA configuration** sets `temporal_resolution: "month"`
3. **Seasonal patterns** can be modeled by varying amounts across months
4. The optimizer accounts for seasonal supply variations when meeting demand

This enables more realistic modeling of:
- Renewable energy seasonality
- Agricultural cycles
- Seasonal water availability
- Monthly demand patterns